In [1]:
import pandas as pd
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
# Set options to show all columns
pd.set_option('display.max_columns', None)

In [2]:
# Upload csv file : file is copied from "C:\repos\immo-eliza-team6-analysis\analysis\Team_6_Step_3b_outliers_cat.ipynb"
# after running "analysis\Team_6_Step_3b_outliers_cat.ipynb" without the coding blocks for 
# "Encoding - label encoding" on kitchen type, state of building and EPC 
# (hence, missing values on categorical values was treated)
data = r'raw.csv' 
df = pd.read_csv(data, sep = ',')

In [3]:
### HANDLING CATEGORICAL DATA ###

#Label encoding for EPC from A to F replaced by from 1 to 6
order = ['A', 'B', 'C', 'D', 'E', 'F']
category_mapping = {category: rank for rank, category in enumerate(order, start=1)}
df['EPC_encoded'] = df['epc'].map(category_mapping)
df=df.drop(['epc'],axis = 1)

#One hot encoding for kitchen type
dummies = pd.get_dummies(df['Kitchen_type'], prefix='Kitchen')
df=pd.concat([df, dummies], axis = 1)
df=df.drop(['Kitchen_type','Kitchen_Not installed'],axis = 1)

#One hot encoding for province (when no province as True => Brussels)
dummies = pd.get_dummies(df['Province'])
df=pd.concat([df, dummies], axis = 1)
df=df.drop(['Province','Brussels'],axis = 1)
display(df)

,id,locality_name,Postal_code,Price,Subtype,Number_of_bedrooms,Living_area,street,number,latitude,longitude,Open_fire,Swimming_Pool,hasTerrace,terraceSurface,gardenSurface,Number_of_facades,State_of_building,Furnished,landSurface,price_per_sqm,price_per_sqm_land,Assigned_City,Has_Assigned_City,Assigned_City_5,Has_Assigned_City_5,Assigned_City_10,Has_Assigned_City_10,Assigned_City_15,Has_Assigned_City_15,EPC_encoded,Kitchen_Hyper equipped,Kitchen_Installed,Kitchen_Semi equipped,Antwerp,Brabant_Wallon,East Flanders,Flemish Brabant,Hainaut,Limburg,Liège,Luxembourg,Namur,West Flanders
0,20252354,Ronse,9600,319000,House,3,125,Rotterij,148,50.744176,3.625722,False,False,True,23,370,4,To renovate,False,767,2552.000000,415.906128,NaN,False,NaN,False,NaN,False,NaN,False,6,False,True,False,False,False,True,False,False,False,False,False,False,False
1,20251003,Geraardsbergen,9500,299999,House,3,167,Pirrestraat,17,50.752121,3.925495,False,False,True,23,895,2,Good,False,1050,1796.401198,285.713333,NaN,False,NaN,False,NaN,False,Aalst,True,4,False,True,False,False,False,True,False,False,False,False,False,False,False
2,20252002,Mechelen,2800,275000,House,3,154,Caputsteenstraat,150,51.033269,4.491795,False,False,True,10,20,2,To renovate,False,120,1785.714286,2291.666667,NaN,False,NaN,False,NaN,False,Brussels,True,5,False,False,True,True,False,False,False,False,False,False,False,False,False
3,20252352,Gooik,1755,295000,House,3,172,Strijlandstraat,45,50.791877,4.084293,False,False,True,23,300,3,To renovate,False,309,1715.116279,954.692557,NaN,False,NaN,False,NaN,False,Aalst,True,6,False,True,False,False,True,False,False,False,False,False,False,False,False
4,20251715,Dilbeek,1700,715000,House,3,280,Herdebeekstraat 94-94A,0,50.840610,4.228425,False,True,True,36,374,3,As new,False,374,2553.571429,1911.764706,NaN,False,NaN,False,Brussels,True,Brussels,True,3,False,True,False,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,20194622,Nivelles,1400,295000,House,4,135,Rue François Lebon,25,50.600371,4.332420,False,False,True,26,220,2,Good,False,120,2185.185185,2458.333333,NaN,False,NaN,False,NaN,False,Brussels,True,4,False,True,False,False,True,False,False,False,False,False,False,False,False
4176,20157543,Tielt,8700,295000,House,4,169,Blauwvoetstraat,5,50.982288,3.329582,False,False,True,23,220,3,Good,False,412,1745.562130,716.019417,NaN,False,NaN,False,NaN,False,Bruges,True,5,False,True,False,False,False,False,False,False,False,False,False,False,True
4177,20140297,Vilvoorde,1800,299000,House,3,237,Marius Duchéstraat,169,50.934037,4.418561,False,False,True,18,220,3,Good,False,197,1261.603376,1517.766497,NaN,False,NaN,False,Brussels,True,Brussels,True,6,False,False,True,False,True,False,False,False,False,False,False,False,False
4178,20125321,"Saint-Hubert, Mirwart",6870,299000,House,4,215,Rue Du Staplisse,4,50.056322,5.266147,False,False,True,23,220,3,Good,False,386,1390.697674,774.611399,NaN,False,NaN,False,NaN,False,NaN,False,3,False,True,False,False,False,False,False,False,False,False,True,False,False


In [4]:
df2 = df.drop(['locality_name', 'Postal_code','street', 'number', 'Subtype','latitude','longitude','Open_fire','Swimming_Pool', 'hasTerrace','terraceSurface', 'gardenSurface', 'Number_of_facades','Furnished','price_per_sqm',
       'price_per_sqm_land','State_of_building', 'Assigned_City','Assigned_City_5', 'Has_Assigned_City_5', 'Assigned_City_10','Has_Assigned_City_10', 'Assigned_City_15', 'Has_Assigned_City_15'],axis=1)

print(type(df2))
df2.info()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4180 entries, 0 to 4179
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   id                      4180 non-null   int64
 1   Price                   4180 non-null   int64
 2   Number_of_bedrooms      4180 non-null   int64
 3   Living_area             4180 non-null   int64
 4   landSurface             4180 non-null   int64
 5   Has_Assigned_City       4180 non-null   bool 
 6   EPC_encoded             4180 non-null   int64
 7   Kitchen_Hyper equipped  4180 non-null   bool 
 8   Kitchen_Installed       4180 non-null   bool 
 9   Kitchen_Semi equipped   4180 non-null   bool 
 10  Antwerp                 4180 non-null   bool 
 11  Brabant_Wallon          4180 non-null   bool 
 12  East Flanders           4180 non-null   bool 
 13  Flemish Brabant         4180 non-null   bool 
 14  Hainaut                 4180 non-n

In [5]:
df2.shape

(4180, 20)

In [6]:
#splitting in input and output
X = df2.drop(['id', 'Price'], axis=1)
y = df2["Price"]
print(type(X), type(y))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


In [7]:
#splitting in training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11)


In [25]:
X.info()
all = X.columns
print(all)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4180 entries, 0 to 4179
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Number_of_bedrooms      4180 non-null   int64
 1   Living_area             4180 non-null   int64
 2   landSurface             4180 non-null   int64
 3   Has_Assigned_City       4180 non-null   bool 
 4   EPC_encoded             4180 non-null   int64
 5   Kitchen_Hyper equipped  4180 non-null   bool 
 6   Kitchen_Installed       4180 non-null   bool 
 7   Kitchen_Semi equipped   4180 non-null   bool 
 8   Antwerp                 4180 non-null   bool 
 9   Brabant_Wallon          4180 non-null   bool 
 10  East Flanders           4180 non-null   bool 
 11  Flemish Brabant         4180 non-null   bool 
 12  Hainaut                 4180 non-null   bool 
 13  Limburg                 4180 non-null   bool 
 14  Liège                   4180 non-null   bool 
 15  Luxembourg           

In [9]:
#standardize integer-columns except id and price(ouput to be forecasted)
standard_scaler = StandardScaler()
columns = ['Number_of_bedrooms', 'Living_area','landSurface','EPC_encoded'  ]
X_train[columns] = standard_scaler.fit_transform(X_train[columns])
X_test[columns]= standard_scaler.transform(X_test[columns])


In [13]:
#train linear regression model and show score
reg = LinearRegression()
reg.fit(X_train, y_train)
reg.score(X_train, y_train)


0.6129849775068688

In [15]:
#test the model
y_pred = reg.predict(X_test)
reg.score(X_test,y_test)


0.6263851614326752

In [33]:
print("'base'-price :", reg.intercept_)

"coeff :", list(zip(all, reg.coef_))

'base'-price : 502038.33264982083


('coeff :',
 [('Number_of_bedrooms', np.float64(7146.8986980887685)),
  ('Living_area', np.float64(36551.054990573604)),
  ('landSurface', np.float64(36795.68061272313)),
  ('Has_Assigned_City', np.float64(30394.31828966215)),
  ('EPC_encoded', np.float64(-53682.41368381156)),
  ('Kitchen_Hyper equipped', np.float64(53796.55035383093)),
  ('Kitchen_Installed', np.float64(27232.668430188147)),
  ('Kitchen_Semi equipped', np.float64(7596.203072122257)),
  ('Antwerp', np.float64(-154604.8327814379)),
  ('Brabant_Wallon', np.float64(-128195.99851779251)),
  ('East Flanders', np.float64(-198459.36736635966)),
  ('Flemish Brabant', np.float64(-157884.98143869208)),
  ('Hainaut', np.float64(-270239.7377553209)),
  ('Limburg', np.float64(-214600.1972264479)),
  ('Liège', np.float64(-240990.8544619341)),
  ('Luxembourg', np.float64(-235385.73613678737)),
  ('Namur', np.float64(-279692.07624935254)),
  ('West Flanders', np.float64(-218515.25708532488))])